# Query and Visualise Available Satellite Data

This notebook presents a workflow to query and visualise the Sentinel-2 and Landsat data available in Google Earth Engine under an area of interest (AOI) and over a specified time range.

In [ ]:
# Necessary Python modules
import sys
import os
os.environ['USE_PYGEOS'] = '0'

from ml4floods.data import utils
import geopandas as gpd
from georeader.readers import ee_query, scihubcopernicus_query
import folium
from datetime import datetime, timezone, timedelta
import ee
import geopandas as gpd
import pandas as pd
import folium
from zoneinfo import ZoneInfo
import geemap.foliumap as geemap
from georeader.readers import query_utils
import folium
from georeader.readers import S2_SAFE_reader

# Set bucket will not be requester pays
utils.REQUESTER_PAYS_DEFAULT = False

from dotenv import load_dotenv

## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [ ]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

# Uncomment for alternative version for Windows (r"" indicates raw string)
#env_file_path = r"C:/Users/User/floodmapper/.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] Failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] Missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] Missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] Missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] Base path does not exist: \n{base_path} "
bucket_name = os.environ["BUCKET_URI"]
assert bucket_name is not None and bucket_name != "", f"Bucket name not defined {bucket_name}"

print("[INFO] Successfully loaded FloodMapper environment.")

**Set the details of the event and mapping session here**.

In [ ]:
# All work is conducted under a unique session name
session_name = "EMSR570"

# Flooding date range (UTC)
# May need to start day or two earlier
flood_start_date = "2022-02-23"
flood_end_date = "2022-03-20"

# Pre-flood date range
# This is a time period before the flood event to inspect reference data
preflood_start_date = "2022-02-10"
preflood_end_date = "2022-02-20"

## Parse and check date information

We assume the UTC timezone for all date queries.

In [ ]:
# First parse the pre- and post-flood dates
tz = ZoneInfo("UTC")

_start = datetime.strptime(flood_start_date,"%Y-%m-%d").replace(tzinfo=tz)
_end = datetime.strptime(flood_end_date,"%Y-%m-%d").replace(tzinfo=tz)
flood_start_period, flood_end_period = sorted([_start, _end])
flood_duration = flood_end_period - flood_start_period
print(f"[INFO] Flood search period: \n\t{flood_start_period} to \n\t{flood_end_period}")
print(f"[INFO] Flood duration = {flood_duration}\n")

_start = datetime.strptime(preflood_start_date,"%Y-%m-%d").replace(tzinfo=tz)
_end = datetime.strptime(preflood_end_date,"%Y-%m-%d").replace(tzinfo=tz)
preflood_start_period, preflood_end_period = sorted([_start, _end])
preflood_duration = preflood_end_period - preflood_start_period
print(f"[INFO] Pre-flood search period: \n\t{preflood_start_period} to \n\t{preflood_end_period}")
print(f"[INFO] Pre-flood duration = {preflood_duration}\n")
margin = flood_start_period - preflood_end_period
print(f"[INFO] Margin before flood = {margin}\n")

## Load the gridded AOIs to be mapped

Here we load the gridded AoIs from the GCP bucket. We created this file in the previous notebook.

In [ ]:
# Gridded AoI filename
grid_aoi_file = "patches_to_map.geojson"

# Form the session path and output path on the GCP bucket
session_path = os.path.join(bucket_name, "0_DEV/1_Staging/operational", session_name).replace("\\", "/")
grid_aoi_path = os.path.join(session_path, grid_aoi_file).replace("\\", "/")
grid_aois = utils.read_geojson_from_gcp(grid_aoi_path)
grid_aois.drop_duplicates(inplace=True)
print(f"[INFO] Loaded gridded_aois from the following file:\n\t{grid_aoi_path}")
grid_aois

In [ ]:
# Merge the grid patches to form an outline (MultiPolygon)
aoi_outline_df = grid_aois.geometry.unary_union
aoi_outline_gdf = gpd.GeoDataFrame(geometry=[aoi_outline_df], crs="EPSG:4326")

# Plot the grid and outline on a Leaflet map
m = grid_aois.explore(style_kwds={"fillOpacity": 0.3, "weight": 0.0}, name="Grid Patches")
aoi_outline_gdf.explore(m=m, color="red", style_kwds={"fillOpacity": 0.0, "weight": 2.0}, 
                        name="AoI Outline", highlight=False)
folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
# Check if any grid patches are duplicates
are_duplicates = grid_aois.duplicated().any()
print(f"Are any grid names duplicates? -> {are_duplicates}")

In [ ]:
# Drop any duplicates now
grid_aois.drop_duplicates(inplace=True)
grid_aois.duplicated().any()

## Query what images are available in Google Earth Engine

In [ ]:
# Initialise the Google Earth Engine connection.
# Follow instructions on login prompt, if required.
ee.Authenticate()

In [ ]:
# Initialise the GEE project
ee.Initialize()

In [ ]:
%%time

# Run a GEE query for Landsat and Sentinel-2 data.
#  producttype can be 'both', 'S2', "Landsat", "L8" or "L9".
#  add_s2cloudless adds a column that indicates if the s2cloudless image is available .
flood_images_gee, flood_collection = ee_query.query(
    area=aoi_outline_df, 
    date_start=flood_start_period, 
    date_end=flood_end_period,                                                   
    producttype="both", 
    return_collection=True, 
    add_s2cloudless=True)

# Print data about the available images
num_images = flood_images_gee.shape[0]
print(f"[INFO] Found {num_images} flooding images on archive.")

In [ ]:
# Run the same query for the pre-flood period
preflood_images_gee, preflood_collection = ee_query.query(
    area=aoi_outline_df, 
    date_start=preflood_start_period, 
    date_end=preflood_end_period,                                                   
    producttype="both", 
    return_collection=True, 
    add_s2cloudless=True)
num_images = preflood_images_gee.shape[0]
print(f"[INFO] Found {num_images} pre-flood images on archive.")

## Visualise the available Landsat and S2 imagery

Here we can directly visualise the imagery for each satellite overpass. This will help make a selection on which days to include in the flood mapping operation.

Once the map loads, click on individual Satellite + Date combinations to show the imagery. 

**Note: the map and imagery can take a few seconds to load.**

In [ ]:
%%time
# Intialise the OpenStreetMap base layer
m = geemap.Map(location=aoi_outline_df.centroid.coords[0][-1::-1], zoom_start=8)

# Add the pre-flood data
for (day, satellite), images_day in preflood_images_gee.groupby(["solarday", "satellite"]):    
    image_col_day_sat = preflood_collection.filter(ee.Filter.inList("title", images_day.index.tolist()))    
    bands = ["B11","B8","B4"] if satellite.startswith("S2") else ["B6","B5","B4"] 
    m.addLayer(image_col_day_sat, 
               {"min":0, "max":3000 if satellite.startswith("S2") else 0.3, "bands": bands},
               f"{satellite}: {day}",
               False)
    
# Add the flooding data
for (day, satellite), images_day in flood_images_gee.groupby(["solarday", "satellite"]):    
    image_col_day_sat = flood_collection.filter(ee.Filter.inList("title", images_day.index.tolist()))    
    bands = ["B11","B8","B4"] if satellite.startswith("S2") else ["B6","B5","B4"]
    m.addLayer(image_col_day_sat, 
               {"min":0, "max":3000 if satellite.startswith("S2") else 0.3, "bands": bands},
               f"{satellite}: {day}",
               False)

aoi_outline_gdf.explore(style_kwds={"fillOpacity": 0.1}, color="black", name="AoI", highlight=False, m=m)
folium.LayerControl(collapsed=False).add_to(m)

In [ ]:
# Show the map
m

For the EMSR570 flood, we can see the following days provide good imagery:

**Reference Period:**
 * S2A: 2022-02-11
 * LC08: 2022-02-12
 * LC09: 2022-02-13
 * S2A: 2022-02-18
 * S2B: 2022-02-19
 
**Flood Period:**
 * LC08: 2022-02-28
 * S2B: 2022-03-01
 * LC08: 2022-03-02
 * LC09: 2022-03-03
 * S2A: 2022-03-03
 * LC08: 2022-03-04
 * S2B: 2022-03-05
 * S2A: 2022-03-06
 * LC09: 2022-03-08
 * S2B: 2022-03-09
 * LC09: 2022-03-10